In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
df = pd.read_csv("/gdrive/MyDrive/titanic.csv")

In [ ]:
df.head()

,Unnamed: 0,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [ ]:
df.tail()

,Unnamed: 0,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
886,886,887,"Montvila, Rev. Juozas",2,male,27.0,0,0,211536,13.00,NaN,S,0
887,887,888,"Graham, Miss. Margaret Edith",1,female,19.0,0,0,112053,30.00,B42,S,1
888,888,889,"Johnston, Miss. Catherine Helen ""Carrie""",3,female,NaN,1,2,W./C. 6607,23.45,NaN,S,0
889,889,890,"Behr, Mr. Karl Howell",1,male,26.0,0,0,111369,30.00,C148,C,1
890,890,891,"Dooley, Mr. Patrick",3,male,32.0,0,0,370376,7.75,NaN,Q,0


**Data Exploration**

In [ ]:
#Here we will drop unrelevant columns like name,passendgerId,parch etc
#As they do not impact on the Survival rate
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'], axis='columns',inplace=True)
df.head()


,Unnamed: 0,Pclass,Sex,Age,Fare,Survived
0,0,3,male,22.0,7.2500,0
1,1,1,female,38.0,71.2833,1
2,2,3,female,26.0,7.9250,1
3,3,1,female,35.0,53.1000,1
4,4,3,male,35.0,8.0500,0


In [ ]:
#In this dataset Survived is the dependent variable and Pclass,Sex,Age and Fare are independent variables
#Now we will seperate the dependent(target) variable and independent(input) variable into two seperate entities
target = df.Survived
inputs = df.drop('Survived',axis='columns')

In [ ]:
#in the above dataset Sex column is text, as the machine can understand only numbers we will convert the Sex column into 0 or 1 using the One-hot encoding method
dummies = pd.get_dummies(inputs.Sex)
dummies.head(5)

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [ ]:
#Now we will add this dummies column to the inputs dataframe by using the pandas 'concat' function
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(5)

,Unnamed: 0,Pclass,Sex,Age,Fare,female,male
0,0,3,male,22.0,7.2500,0,1
1,1,1,female,38.0,71.2833,1,0
2,2,3,female,26.0,7.9250,1,0
3,3,1,female,35.0,53.1000,1,0
4,4,3,male,35.0,8.0500,0,1


In [ ]:
inputs.drop('Sex',axis='columns',inplace=True)
inputs.head(5)

,Unnamed: 0,Pclass,Age,Fare,female,male
0,0,3,22.0,7.2500,0,1
1,1,1,38.0,71.2833,1,0
2,2,3,26.0,7.9250,1,0
3,3,1,35.0,53.1000,1,0
4,4,3,35.0,8.0500,0,1


In [ ]:
#To know whether there are any NaN values in the inputs dataframe
#we use the isna().any() function
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [ ]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [ ]:
#Now we will fill the NaN values withe average value using the mean()function
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head(7)

,Unnamed: 0,Pclass,Age,Fare,female,male
0,0,3,22.000000,7.2500,0,1
1,1,1,38.000000,71.2833,1,0
2,2,3,26.000000,7.9250,1,0
3,3,1,35.000000,53.1000,1,0
4,4,3,35.000000,8.0500,0,1
5,5,3,29.699118,8.4583,0,1
6,6,1,54.000000,51.8625,0,1


In [ ]:
#Now we will use the Sklearn train, test split method before training our model
#to confirm the model is not baised


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(inputs,target,test_size=0.2)
#here we are dividing the train and test data into 80% and 20%

In [ ]:
len(X_train)
#80% of the inputs data

712

In [ ]:
len(X_test)
#20% of the inputs data

179

In [ ]:
len(inputs)
#length of total inputs data

891

**Naives Bayes**

In [ ]:
#we willimport the gaussian naive bayes from the sklearn library
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

**Training the Model**

In [ ]:
#train the model using the fit method
model.fit(X_train,y_train)

GaussianNB()

**Measure the Accuracy**

In [ ]:
model.score(X_test,y_test)
#score will change as it depends on the sample size

0.7988826815642458

In [ ]:
#check the samples of X_test
X_test[:10]

,Unnamed: 0,Pclass,Age,Fare,female,male
732,732,2,29.699118,0.0000,0,1
386,386,3,1.000000,46.9000,0,1
96,96,1,71.000000,34.6542,0,1
651,651,2,18.000000,23.0000,1,0
383,383,1,35.000000,52.0000,1,0
775,775,3,18.000000,7.7500,0,1
840,840,3,20.000000,7.9250,0,1
363,363,3,35.000000,7.0500,0,1
453,453,1,49.000000,89.1042,0,1
223,223,3,29.699118,7.8958,0,1


In [ ]:
#check the y_test smaples
y_test[:10]
#this samples also depends on the random sample size, as we execute the model again it's values change

732    0
386    0
96     0
651    1
383    1
775    0
840    0
363    0
453    1
223    0
Name: Survived, dtype: int64

In [ ]:
#predict the Survival rate
model.predict(X_test[0:10])

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
model.predict_proba(X_test[:10])

array([[0.97269891, 0.02730109],
       [0.96629874, 0.03370126],
       [0.83852254, 0.16147746],
       [0.01762838, 0.98237162],
       [0.00392547, 0.99607453],
       [0.98565915, 0.01434085],
       [0.98656578, 0.01343422],
       [0.98691888, 0.01308112],
       [0.65257455, 0.34742545],
       [0.9869718 , 0.0130282 ]])

**Calculating the Score using the Cross-validation**

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train,y_train,cv=5)

array([0.75524476, 0.74125874, 0.8028169 , 0.8028169 , 0.78873239])